# CS483 - Colab 4
## Collaborative Filtering

In [1]:
from IPython.display import Image
print("Colab 4 Mascot")
Image(url='https://media4.giphy.com/media/wtdVYmaRWJ1PyPcc8e/giphy.gif?cid=ecf05e478jdbzo92b89f2b5ud5184xda9cen1xce4bwmjeyu&ep=v1_stickers_search&rid=giphy.gif&ct=s',width=150)

Colab 4 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package libxtst6:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libxtst6_2%3a1.2.3-1build4_amd64.deb ...
Unpacking libxtst6:amd64 (2:1.2.3-1build4) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../openjdk-8-jre-headless_8u422-b05-1~22.04_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u422-b05-1~22.04) ...
Sel

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1QtPy_HuIMSzhtYllT3-WeM3Sqg55wK_D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.training')

id='1ePqnsQTJRRvQcBoF2EhoPU8CU1i5byHK'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.test')

id='1ncUBWdI5AIt3FDUJokbMqpHD2knd5ebp'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('MovieLens.item')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Data Loading

In this Colab, we will be using the [MovieLens dataset](https://grouplens.org/datasets/movielens/), specifically the 100K dataset (which contains in total 100,000 ratings from 1000 users on ~1700 movies).

We load the ratings data in a 80%-20% ```training```/```test``` split, while the ```items``` dataframe contains the movie titles associated to the item identifiers.

In [7]:
schema_ratings = StructType([
    StructField("user_id", IntegerType(), False),
    StructField("item_id", IntegerType(), False),
    StructField("rating", IntegerType(), False),
    StructField("timestamp", IntegerType(), False)])

schema_items = StructType([
    StructField("item_id", IntegerType(), False),
    StructField("movie", StringType(), False)])

training = spark.read.option("sep", "\t").csv("MovieLens.training", header=False, schema=schema_ratings)
test = spark.read.option("sep", "\t").csv("MovieLens.test", header=False, schema=schema_ratings)
items = spark.read.option("sep", "|").csv("MovieLens.item", header=False, schema=schema_items)

In [8]:
training.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- item_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: integer (nullable = true)



In [9]:
items.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- movie: string (nullable = true)



### Your task

Let's compute some stats!  What is the number of ratings in the training and test dataset? How many movies are in our dataset?

In [10]:
''' 3 lines of code in total expected.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.sql import functions as F
training.where(F.col("rating").isNotNull()).select(F.count("rating")).show()
test.where(F.col("rating").isNotNull()).select(F.count("rating")).show()

+-------------+
|count(rating)|
+-------------+
|        80000|
+-------------+

+-------------+
|count(rating)|
+-------------+
|        20000|
+-------------+



Using the training set, train a model with the Alternating Least Squares method available in the Spark MLlib: [https://spark.apache.org/docs/latest/ml-collaborative-filtering.html](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html)

In [11]:
''' 5-6 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.recommendation import ALS
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="item_id", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

Now compute the RMSE on the test dataset.


In [12]:
''' 4-5 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
from pyspark.ml.evaluation import RegressionEvaluator
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.1267999197333132


At this point, you can use the trained model to produce the top-K recommendations for each user.

In [47]:
''' 9-10 lines of code in total expected but can differ based on your style.
For sub-parts of the question (if any), creating different cells of code would be recommended.'''
# YOUR CODE HERE
k = 5
recs = model.recommendForAllUsers(k)
res = recs.rdd \
  .flatMap(lambda x: [(x[0], a.item_id, a.rating) for a in x[1]]) \
  .toDF(["user_id", "item_id", "rating"]) \
  .join(items, "item_id") \
  .groupBy("user_id") \
  .agg(F.collect_list("movie").alias("movies")) \
  .toDF("user_id", "movies")

res.show()

+-------+--------------------+
|user_id|              movies|
+-------+--------------------+
|      1|[8 Heads in a Duf...|
|      2|[Tough and Deadly...|
|      3|[Portrait of a La...|
|      4|[Tough and Deadly...|
|      5|[Chungking Expres...|
|      6|[Oscar & Lucinda ...|
|      7|[Angel Baby (1995...|
|      8|[Replacement Kill...|
|      9|[I'm Not Rappapor...|
|     10|[Safe (1995), Ang...|
|     11|[Beautiful Thing ...|
|     12|[8 Heads in a Duf...|
|     13|[Oscar & Lucinda ...|
|     14|[Tough and Deadly...|
|     15|[Mixed Nuts (1994...|
|     16|[City of Industry...|
|     17|[Alphaville (1965...|
|     18|[Tough and Deadly...|
|     19|[Tough and Deadly...|
|     20|[Stupids, The (19...|
+-------+--------------------+
only showing top 20 rows



Once you have working code for each cell above, **head over to Gradescope, carefully read the questions, and submit your solution for this Colab**!